### IMDB Logistic Regression

Long IMDB moview reviews. Positive: 25000, Negative: 25000

Accuracy: 0.868

In [2]:
import pickle
import re
import os

os.chdir('/Users/Bya/Dropbox/Research/datas/movieclassifier/')
from vectorizer import vect


clf = pickle.load(open(
        os.path.join('pkl_objects',
                     'classifier.pkl'), 'rb'))

In [23]:
import numpy as np

label = {0: 'negative', 1:'positive'}

example = ['I love this movie']

X = vect.transform(example)

print('Prediction: %s\nProbability: %.2f%%' % \
     (label[clf.predict(X)[0]],
     np.max(clf.predict_proba(X)) * 100))

clf.predict_proba(X)[0][1]

Prediction: positive
Probability: 83.03%


0.83032271725605766

In [15]:
clf.predict_proba(X)

array([[ 0.49762755,  0.50237245]])

### IMDB Naive Bayes

Short IMDB moview reviews. Positive: 5000, Negative: 50000

In [4]:
with open("/Users/Bya/git/predictEPL/NLTK/NLTK_TUTORIAL/pickled_algos/naiveBayes_for_short_reviews_NEW.pickle", "rb") as saved_classifier_f:
    nb_classifier = pickle.load(saved_classifier_f)

In [6]:
tweets = [
"Great start to the evening seeing Mo Farah win the 5000m, now for #Arsenal. #Beijing2015 #NEWARS http://t.co/9ulMYqQ91M",
"Good squad there. Could be in with a very small chance #nufc",
"Blog: Newcastle United feedback: Will the Magpies replace like for like if they sell? http://t.co/rxfXZbGwVE #NUFC",
"FourFourTwo: Knee injury sidelines Ozil http://t.co/rxfXZbGwVE #NUFC",
"Blog: Newcastle United coach Steve Black has got in spot on once again http://t.co/rxfXZbGwVE #NUFC",
"RT @ghanafans: KICK OFF - #Arsenal will be attacking from right to left #NUFCvAFC  #BPL",
"I hate that andre marraner he is a total cock #nufc",
"Really dirty stuff from Sissoko, stomping on Nacho Monreal's ankle. Only a yellow but could have been a sending off. #Arsenal #NUFC",
"Really please to make my debut for this fantastic club. Great to get the win. Now we focus on Saturdays game #toonarmy #NUFC",
"RT @TheSunFootball: #Arsenal break well through Sanchez and Oxlade-Chamberlain. Terrible return pass from Ramsey wrecks the move. 0-0",
"RT @NUFC_Index: NUFC hunting in packs early on!",
"Gets a yellow for a stamp!#nufc",
"Come on gunners! #NUFCvAFC #Arsenal"]

In [8]:
import os
os.chdir('/Users/Bya/git/predictEPL/MyFunctions/')
import tokenizers
import senti_word_net
import featx
import replacers

from nltk.corpus import stopwords

from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer

In [9]:
def cleanHash(word):
    if word[0] == '#':
        return '#'
    elif word[0] == '@':
        return '@'
    elif word[0:4] == 'http':
        return 'http'
    else:
        return word

In [28]:
example = 1
for tweet in tweets:
    print("Example %d=====================================================\n" % example)
    example += 1

    print("Tweet: \n \n %s \n" % tweet)
    
    # can't -> cannot, bya's -> bya is
    replacer = replacers.RegexpReplacer()
    tweet = replacer.replace(tweet)
    
    # Tweet tokenizer
    words = TweetTokenizer().tokenize(tweet)
    print("Words(tweetTokenizer): \n\n %s \n" % words)
    
    # defining stopwords
    english_stops = set(stopwords.words('english'))
    english_stops_added = english_stops | {'.', ',', ':', ';', '#', '?', 'RT', '-', '@', 'rt'}
    words = [word.lower() for word in words if word.lower() not in english_stops_added]
    
    # Stemmer
    porter = PorterStemmer()
    words = [porter.stem(word) for word in words]
    
    # words = map(lambda word: cleanHash(word), words)
    words = [cleanHash(word) for word in words]
    
    # defining stopwords
    english_stops_added =  {'.', ',', '!', ':', ';', '#', '?', 'RT', '-', '@', 'rt', 'http'}
    words = [word.lower() for word in words if word.lower() not in english_stops_added]
 
    print("Words(byaCleaner): \n \n %s \n" % words)

    print("\n-------------Naive Bayes: Short Reviews--------------------\n")
    
    probs = nb_classifier.prob_classify(featx.bag_of_words(words))
    print("Short Reviews Sentiment: %s" % probs.max())
    print("\t pos: %s" % probs.prob('pos'))
    print("\t neg: %s" % probs.prob('neg'))
    
    
    print("\n------------Logictic Regression: Long Reviews---------------\n")

    label = {0: 'neg', 1:'pos'}

    sentence = [' '.join(words)]

    X = vect.transform(sentence)

    print("Long Reviews Sentiment: %s" % label[clf.predict(X)[0]])
    print("\t pos: %s" % clf.predict_proba(X)[0][1])
    print("\t neg: %s \n\n" % clf.predict_proba(X)[0][0])

Example 1=====================================================

Tweet: 
 
 Great start to the evening seeing Mo Farah win the 5000m, now for #Arsenal. #Beijing2015 #NEWARS http://t.co/9ulMYqQ91M 

Words(tweetTokenizer): 

 ['Great', 'start', 'to', 'the', 'evening', 'seeing', 'Mo', 'Farah', 'win', 'the', '5000m', ',', 'now', 'for', '#Arsenal', '.', '#Beijing2015', '#NEWARS', 'http://t.co/9ulMYqQ91M'] 

Words(byaCleaner): 
 
 ['great', 'start', 'even', 'see', 'mo', 'farah', 'win', '5000m'] 


-------------Naive Bayes: Short Reviews--------------------

Short Reviews Sentiment: pos
	 pos: 0.7715516286811658
	 neg: 0.22844837131883497

------------Logictic Regression: Long Reviews---------------

Long Reviews Sentiment: pos
	 pos: 0.807552071673
	 neg: 0.192447928327 


Example 2=====================================================

Tweet: 
 
 Good squad there. Could be in with a very small chance #nufc 

Words(tweetTokenizer): 

 ['Good', 'squad', 'there', '.', 'Could', 'be', 'in', 'with'